In [6]:
class Person:
    def __init__(self, name, phone_number, address):
        self.name = name
        self.phone_number = phone_number
        self.address = address
    def __str__(self):
         return f"Name: {self.name}, phone_number: {self.phone_number}, address: {self.address}"
class Customers(Person):
    def __init__(self, name, phone_number, address):
        super().__init__(name, phone_number, address)
    def __str__(self):
         return f"Name: {self.name}, phone_number: {self.phone_number}, address: {self.address}"
class Admins(Person):
    def __init__(self, name, phone_number, address, username, password):
        super().__init__(name, phone_number, address)
        self.username = username
        self.password = password
    def __str__(self):
         return f'''Name: {self.name}, phone_number: {self.phone_number}, address: {self.address}, 
                    Username: {self.username}, Password: {self.password}'''
class Products:
    def __init__(self, pd_name, quantity, price ):
        self.pd_name = pd_name
        self.quantity = quantity
        self.price = price
    def __str__(self):
         return f"Name: {self.pd_name}, quantity: {self.quantity}, price: {self.price}"
class Orders:
    def __init__(self, item, quantity, datetime, total):
        self.item = item
        self.quantity = quantity
        self.datetime = datetime
        self.total = total
    def __str__(self):
        return f"item: {self.item}, quantity: {self.quantity}, datetime: {self.datetime}, total: {self.total}"

In [85]:
def active_btn(btn):
    btn.configure(bg = "#006fff")
def clear():
    name.delete(0, "end")
    quantity.delete(0, "end")
    price.delete(0, "end")
def update(rows):
    tree.delete(*tree.get_children())
    for i in rows:
            tree.insert('','end',iid=i[0], value = i)
def load_data():
    query = "SELECT * FROM products"
    cursor = mydb.cursor()
    cursor.execute(query)
    rows = cursor.fetchall()
    update(rows)
def add_product():
    product_name = name.get()
    product_quantity = quantity.get()
    product_price = price.get()

    if product_name == "" or product_quantity =="" or product_price =="":
        messagebox.showerror("Error!", "Please Enter name, quantity and price!")
    else:
        new_product = Products(product_name, int(product_quantity), int(product_price))
        cursor = mydb.cursor()
        cursor.execute("INSERT INTO products(productName, quantity, price) VALUES (?, ?, ?)", 
                       (new_product.pd_name, new_product.quantity, new_product.price))
        mydb.commit()
        load_data()
        clear()
def del_product():
    if messagebox.askyesno("Confirm?", "Are you sure?"):
        selected = tree.selection()[0]
        query = 'DELETE FROM products WHERE productid = ?'
        values = (selected,)
        cursor = mydb.cursor()
        cursor.execute(query,values)
        mydb.commit()
        load_data()
    else:
        return True

def search_product():
    product_name = search_entry.get()
    query = "SELECT * FROM products WHERE productName LIKE '%"+product_name+"%'"
    cursor = mydb.cursor()
    cursor.execute(query)
    rows = cursor.fetchall()
    update(rows)
def show_all():
    load_data()
    clear()
    
def show_detail():
    item = tree.item(tree.focus())
    name.insert(0,item['values'][1])
    quantity.insert(0,item['values'][2])
    price.insert(0,item['values'][3])
    btn_save_update.configure(state = 'active')
    btn_show_detail.configure(state = 'disable')

def update_product():
    product_name = name.get()
    new_product_quantity = quantity.get()
    new_product_price = price.get()
    query = 'UPDATE products SET price = ?, quantity = ? WHERE productName = ?'
    cursor = mydb.cursor()
    cursor.execute(query,(new_product_price, new_product_quantity, product_name))
    mydb.commit()
    load_data()
    clear()
    btn_save_update.configure(state = 'disable')
    btn_show_detail.configure(state = 'active')
    show_all()
    
def chose():
    item = tree.item(tree.focus())
    pd_name.insert(0,item['values'][1])

def add_to_bill():
    item_chose = []
    item = tree.item(tree.focus())
    product_name = item['values'][1]
    product_price =  item['values'][3]
    quantity_item_select = quantity_buy.get()
    
    tt_quantity = int(total_quantity.get()) + int(quantity_item_select)
    sum_item_price = int(product_price) * int(quantity_item_select)
    tt_price = sum_item_price + int(total_price.get())
    
    total_quantity.delete(0,"end")
    total_price.delete(0,"end")
    
    total_quantity.insert(0,tt_quantity)
    total_price.insert(0,tt_price) 
    product_item = (product_name, product_price, quantity_item_select, sum_item_price)
    #ADD TO TREEVIEW
    item_chose.append(product_item)
    for i in item_chose:
        tree_order.insert('','end',iid=i[0], value = i)
    product_name = item['values'][1]
    quantity_item_select = quantity_buy.get()
    new_quantity = item['values'][2] - int(quantity_item_select)
    query = 'UPDATE products SET quantity = ? WHERE productName = ?'
    cursor = mydb.cursor()
    cursor.execute(query,(new_quantity, product_name))
    mydb.commit()
    load_data()
    pd_name.delete(0,"end")
    quantity_buy.delete(0,"end")
def return_to_list():
    item = tree_order.item(tree_order.focus())
    quantity_on_bill = item['values'][2] 
    product_name = item['values'][0]
    cursor = mydb.cursor()
    cursor.execute("SELECT quantity FROM products WHERE productName = ?", (product_name,))
    result = cursor.fetchone()
    quantity_org = result[0]
    update_quantity = quantity_on_bill + quantity_org
    update_query = 'UPDATE products SET quantity = ? WHERE productName = ?'
    cursor.execute(update_query,(update_quantity, product_name) )
    mydb.commit()
    load_data()
    selected_item = tree_order.selection()[0]
    tree_order.delete(selected_item)
    total_quantity.delete(0,"end")
    total_price.delete(0,"end")
    total_quantity.insert(0,"0")
    total_price.insert(0,"0")
def create_order():
    product_name_list = []
    current_time = str(datetime.datetime.now())
    tt_quantity = total_quantity.get()
    tt_price = total_price.get()
    
    data = [tree_order.item(child)["values"] for child in tree_order.get_children()]
    for row in data:
        product_name_list.append(row[0])
    new_order = Orders(product_name_list,tt_quantity, current_time, tt_price)
    cursor = mydb.cursor()
    cursor.execute("INSERT INTO orders(item_list, quantity, datetime, total) VALUES (?, ?, ?, ?)", 
                   (str(new_order.item), new_order.quantity, new_order.datetime, new_order.total))
    mydb.commit()
    messagebox.showinfo("Success!", "New order is created!. Check it in order page")
    pd_name.insr(0,"end")
    quantity_buy.delete(0,"end")

In [90]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import datetime
import sqlite3 

def login():
    username = username_entry.get()
    password = password_entry.get()
    if username == "admin" and password == "admin":
        main_app()
        
login_window = tk.Tk()
login_window.title("Login")
login_window.geometry("300x150")

username_label = tk.Label(login_window, text="Username:")
username_label.pack()

username_entry = tk.Entry(login_window)
username_entry.pack()

password_label = tk.Label(login_window, text="Password:")
password_label.pack()

password_entry = tk.Entry(login_window, show="*")
password_entry.pack()

login_button = tk.Button(login_window, text="Login", command=login)
login_button.pack()

login_window.mainloop()
def main_app():
    mydb = sqlite3.connect('test1.db')
    create_query = '''CREATE TABLE IF NOT EXISTS products 
                    (productid INTEGER PRIMARY KEY AUTOINCREMENT, 
                    productName VARCHAR NOT NULL, 
                    quantity INTEGER NOT NULL, 
                    price INTEGER NOT NULL)
                    '''
    create_query1 = '''CREATE TABLE IF NOT EXISTS orders 
                    (orderid INTEGER PRIMARY KEY AUTOINCREMENT, 
                    item_list VARCHAR NOT NULL,
                    quantity INTEGER NOT NULL,
                    datetime DATETIME NOT NULL, 
                    total INTEGER NOT NULL)
                   '''

    cursor = mydb.cursor()
    mydb.execute(create_query)
    mydb.execute(create_query1)

    login_window.destroy()
    app = tk.Tk()
    app.iconbitmap('myIcon.ico')

    def remove_widget():
        for w in app.winfo_children():
            w.destroy()

    def sale_page():
        remove_widget()
        app.title("Sale page")
        app.geometry("1400x600")
        #nav
        nav = tk.Frame(app, bg = '#023047')
        nav.pack(side = tk.TOP)
        nav.pack_propagate(False)
        nav.config(width = 1400, height = 60)
        btn_sale_page = tk.Button(nav, text ="Sale", bd = 0, height = 2, width = 15,
                                  command = sale_page)
        btn_product_page = tk.Button(nav, text ="Product",bd = 0, height = 2, width = 15, 
                                     command = product_page)
        btn_order_page = tk.Button(nav, text ="ORDER", bd=0, height = 2, width = 15, 
                                   command = order_page)

        btn_sale_page.place(x = 20, y = 10)
        btn_product_page.place(x = 200, y = 10)
        btn_order_page.place(x = 380, y = 10)
        #main
        main_frame = tk.Frame(app, highlightthickness = 5, highlightcolor = '#c3c3c3', padx = 20)
        main_frame.pack( fill="both", expand="yes")
        main_frame.pack_propagate(False)
        main_frame.config(width = 1400, height = 540)
        #right frame
        right_frame = tk.Frame(main_frame, highlightthickness = 5, highlightcolor = 'red')
        right_frame.pack(side =tk.RIGHT, fill="both", expand="yes")
        right_frame.pack_propagate(False)
        right_frame.config(width = 700, height = 540)
        #left frame
        left_frame = tk.Frame(main_frame, highlightthickness = 5, highlightcolor = 'red')
        left_frame.pack(side = tk.LEFT, fill="both", expand="yes")
        left_frame.pack_propagate(False)
        left_frame.config(width = 700, height = 540)

        global tree
        columns = ('product_id', 'product_name', 'quantity', 'price')
        tree = ttk.Treeview(left_frame, columns = columns, show = 'headings', height = 10)
        #tree column
        tree.column("#0", width=300)
        tree.column("product_id", width=100)
        tree.column("product_name", width=200)
        tree.column("quantity", width=130)
        tree.column("price", width=150)
        #tree headings
        tree.heading('product_id', text = 'ID')
        tree.heading('product_name', text = 'Product Name')
        tree.heading('quantity', text = 'Quantity')
        tree.heading('price', text = 'Price')
        for i in range(1, 5):
            tree.column("#"+str(i), anchor=tk.CENTER)
        load_data()

        test= tk.LabelFrame(left_frame, text = "CALCULATE")
        global pd_name
        pd_name = tk.Entry(test)
        pd_name.grid()
        q_buy = tk.Label(test, text = "quantity")
        q_buy.grid()
        global quantity_buy
        quantity_buy = tk.Entry(test)
        quantity_buy.grid()
        test.grid(row = 3, column = 2)

        btn_chose= tk.Button(test, text ="CHOSE ITEM", bd=0, bg = '#6296b7', height = 2, width = 15, 
                             command = chose)
        btn_add_bill= tk.Button(test, text ="ADD BILL", bd=0, bg = '#6296b7', height = 2, width = 15, 
                                command = add_to_bill)
        btn_chose.grid( row = 4, column = 3)
        btn_add_bill.grid( row = 4, column = 4)
        global tree_order
        columns = ('product_name', 'price', 'quantity', "total")
        tree_order = ttk.Treeview(right_frame, columns = columns, show = 'headings', height = 10)
        #tree column
        tree_order.column("#0", width=300)
        tree_order.column("product_name", width=200)
        tree_order.column("price", width=150)
        tree_order.column("quantity", width=130)
        tree_order.column("total", width=130)
        #tree headings
        tree_order.heading('product_name', text = 'Product Name')
        tree_order.heading('price', text = 'Price')
        tree_order.heading('quantity', text = 'Quantity')
        tree_order.heading('total', text = 'Total')
        for i in range(1, 5):
            tree_order.column("#"+str(i), anchor=tk.CENTER)

        total_bill = tk.LabelFrame(right_frame, text ="TOTAL INFO")

        global total_quantity
        global total_price

        total_quantity_lbl = tk.Label(total_bill, text = "Total Quantity") 
        total_quantity = tk.Entry(total_bill) 
        total_quantity.insert(0,"0")
        total_price_lbl = tk.Label(total_bill, text = "Total Price") 
        total_price = tk.Entry(total_bill) 
        total_price.insert(0,"0")
        btn_create_order = tk.Button(right_frame, text ="CREATE ORDER", bd=0, bg = '#6296b7',height = 2, width = 15,
                            command = create_order)
        btn_return = tk.Button(right_frame, text ="RETURN ITEM", bd=0, bg = '#6296b7',height = 2, width = 15,
                            command = return_to_list)

        total_bill.grid(row=6, column =2)
        total_quantity_lbl.grid(row=1, column = 1)
        total_quantity.grid(row = 2, column = 1)
        total_price_lbl.grid(row=1, column = 2)
        total_price.grid(row = 2, column = 2)
        btn_create_order.grid( row = 7, column = 2)
        btn_return.grid( row = 7, column = 3)

        tree_order.grid(padx = 15, pady = 15, row =2, column =2, columnspan = 5)
        tree.grid(padx = 15, pady = 15, row =2, column =2, columnspan = 30)

    def product_page():
        remove_widget()
        app.title("Product page")
        app.geometry("1400x600")
        #nav
        nav = tk.Frame(app, bg = '#023047')
        nav.pack(side = tk.TOP)
        nav.pack_propagate(False)
        nav.config(width = 1400, height = 60)
        btn_sale_page = tk.Button(nav, text ="Sale", bd = 0, height = 2, width = 15,
                                  command = sale_page)
        btn_product_page = tk.Button(nav, text ="Product",bd = 0, height = 2, width = 15, 
                                     command = product_page)
        btn_order_page = tk.Button(nav, text ="ORDER", bd=0, height = 2, width = 15, 
                                   command = order_page)
        btn_sale_page.place(x = 20, y = 10)
        btn_product_page.place(x = 200, y = 10)
        btn_order_page.place(x = 380, y = 10)
        #main
        main_frame = tk.Frame(app, highlightthickness = 5, highlightcolor = '#c3c3c3', padx = 20)
        main_frame.pack( fill="both", expand="yes")
        main_frame.pack_propagate(False)
        main_frame.config(width = 1400, height = 540)
        #right frame
        right_frame = tk.Frame(main_frame, highlightthickness = 5, highlightcolor = 'red')
        right_frame.pack(side =tk.RIGHT, fill="both", expand="yes")
        right_frame.pack_propagate(False)
        right_frame.config(width = 700, height = 540)
        #left frame
        left_frame = tk.Frame(main_frame, highlightthickness = 5, highlightcolor = 'red')
        left_frame.pack(side = tk.LEFT, fill="both", expand="yes")
        left_frame.pack_propagate(False)
        left_frame.config(width = 500, height = 540)
        #product-page
        wrapper = tk.LabelFrame(left_frame, text="INFOMATION", height = 400, width = 20)

        lbl_name = tk.Label(wrapper, text ="Product Name")
        lbl_quantity = tk.Label(wrapper, text ="Quantity")
        lbl_price = tk.Label(wrapper, text = "Price")

        global name
        global quantity
        global price
        name = tk.Entry(wrapper)
        quantity = tk.Entry(wrapper)
        price = tk.Entry(wrapper)

        wrapper_btn = tk.LabelFrame( left_frame, text="ACTION ", height = 200, width = 500, bg='#F7F7F7' )
        btn_add = tk.Button( wrapper_btn, text ="ADD NEW", bd=0, bg = '#6296b7',height = 2, width = 15, 
                            command = add_product)
        btn_del = tk.Button( wrapper_btn, text ="DELETE", bd=0, bg = '#d24a46', height = 2, width = 15, 
                            command = del_product )
        btn_update = tk.Button( wrapper_btn, text ="UPDATE", bd=0, bg = '#5DA399', height = 2, width = 15, 
                              )
        global btn_save_update
        btn_save_update = tk.Button( wrapper_btn, text ="SAVE UPDATE", bd=0, bg = '#7FB3D5', 
                                    height = 2, width = 15, state = "disabled", 
                                    command = update_product )
        global btn_show_detail
        btn_show_detail = tk.Button( wrapper_btn, text = "DETAILS", bd=0, bg = '#7FB3D5',
                                    height = 2, width = 15, 
                                    command = show_detail )
        btn_all = tk.Button( wrapper_btn, text = "ALL", bd=0, bg = '#7FB3D5',height = 2, width = 5, 
                            command = show_all)

        btn_search = tk.Button( right_frame, text = "SEARCH", bd=0, bg = '#7FB3D5', height = 2, width = 15, 
                               command = search_product )
        global search_entry
        search_entry = tk.Entry(right_frame)

        #treeview
        global tree
        columns = ('product_id', 'product_name', 'quantity', 'price')
        tree = ttk.Treeview(right_frame, columns = columns, show = 'headings', height = 10)
        #tree column
        tree.column("#0", width=300)
        tree.column("product_id", width=50)
        tree.column("product_name", width=350)
        tree.column("quantity", width=130)
        tree.column("price", width=300)
        #tree hêadings
        tree.heading('product_id', text = 'ID')
        tree.heading('product_name', text = 'Product Name')
        tree.heading('quantity', text = 'Quantity')
        tree.heading('price', text = 'Price')
        for i in range(1, 5):
            tree.column("#"+str(i), anchor=tk.CENTER)
        load_data()

        wrapper.grid(row =1, column =1, sticky ='ew')
        lbl_name.grid( row = 2, column = 1)
        name.grid( row = 3, column = 1)
        lbl_quantity.grid( row = 2, column = 3)
        quantity.grid( row = 3, column = 3)
        lbl_price.grid(row = 2, column = 5)
        price.grid( row = 3, column = 5)
        for widget in wrapper.winfo_children():
            widget.grid_configure(padx = 10, pady=20)    
        wrapper_btn.grid(pady = 10, row =2, column =1,  sticky ='ew')    
        btn_add.grid(row =1, column =1)
        btn_del.grid(row =2, column =1)
        btn_all.grid(row =3, column =1)
        btn_update.grid(row =1, column =2)
        btn_save_update.grid(row =2, column =2)
        btn_show_detail.grid(row =3, column =2)
        for widget in wrapper_btn.winfo_children():
            widget.grid_configure(padx = 50, pady=20)
        btn_search.grid(row =1, column =4)
        search_entry.grid(row = 1, column = 5)
        tree.grid(padx = 15, pady = 15, row =2, column =2, columnspan = 30)

    def order_page():
        remove_widget()
        app.title("Order page")
        app.geometry("1400x600")
        nav = tk.Frame(app, bg = '#023047')
        nav.pack(side = tk.TOP)
        nav.pack_propagate(False)
        nav.config(width = 1400, height = 60)
        btn_sale_page = tk.Button(nav, text ="Sale", bd = 0, height = 2, width = 15,
                                  command = sale_page)
        btn_product_page = tk.Button(nav, text ="Product",bd = 0, height = 2, width = 15, 
                                     command = product_page)
        btn_order_page = tk.Button(nav, text ="ORDER", bd=0, height = 2, width = 15, 
                                   command = order_page)

        btn_sale_page.place(x = 20, y = 10)
        btn_product_page.place(x = 200, y = 10)
        btn_order_page.place(x = 380, y = 10)
        #main
        main_frame = tk.Frame(app, highlightthickness = 5, highlightcolor = '#c3c3c3', padx = 20)
        main_frame.pack( fill="both", expand="yes")
        main_frame.pack_propagate(False)
        main_frame.config(width = 1400, height = 540)

        columns = ('order_id','item_list', 'quantity_item', 'datetime', 'total')
        global tree
        tree = ttk.Treeview(main_frame, columns = columns, show = 'headings', height = 10)
        #tree column
        tree.column("#0", width=300)
        tree.column("order_id", width=350)
        tree.column("item_list", width=200)
        tree.column("quantity_item", width=200)
        tree.column("datetime", width=300)
        tree.column("total", width=200)
        #tree headings
        tree.heading('order_id', text = 'Order ID')
        tree.heading('item_list', text = 'Item List')
        tree.heading('quantity_item', text = 'Quantity Item')
        tree.heading('datetime', text = 'DateTime')
        tree.heading('total', text = 'Total Price')
        #set align center
        for i in range(1, 6):
            tree.column("#"+str(i), anchor=tk.CENTER)
        query = "SELECT * FROM orders"
        cursor = mydb.cursor()
        cursor.execute(query)
        rows = cursor.fetchall()
        update(rows)
        tree.grid(padx = 15, pady = 15, row =2, column =2, columnspan = 30)

    sale_page()
    app.mainloop()    